In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from pyspark.sql.types import StructType, StructField, DoubleType
from pyspark.ml.feature import VectorAssembler, StandardScaler
from pyspark.ml.clustering import KMeans
import urllib.request
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# Initialize Spark Session
spark = SparkSession.builder.appName("Streaming Iris Clustering").getOrCreate()

# Define schema for the streaming data
schema = StructType([
    StructField("sepal_length", DoubleType(), True),
    StructField("sepal_width", DoubleType(), True),
    StructField("petal_length", DoubleType(), True),
    StructField("petal_width", DoubleType(), True)
])

# Download dataset to /tmp/
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data"
save_path = "/tmp/iris.csv"
urllib.request.urlretrieve(url, save_path)
print(f"Downloaded dataset to {save_path}")

# Copy file to DBFS
dbutils.fs.mkdirs("dbfs:/mnt/data/streaming_iris")
dbutils.fs.cp("file:/tmp/iris.csv", "dbfs:/mnt/data/streaming_iris/iris.csv")

# Define streaming data source with schema
streaming_df = spark.readStream.format("csv") \
    .option("header", "false") \
    .schema(schema) \
    .load("dbfs:/mnt/data/streaming_iris/")

# Select features for clustering
feature_cols = ["sepal_length", "sepal_width", "petal_length", "petal_width"]
assembler = VectorAssembler(inputCols=feature_cols, outputCol="raw_features")
static_df = spark.read.format("csv").option("header", "false").schema(schema).load("dbfs:/mnt/data/streaming_iris/iris.csv")
static_features = assembler.transform(static_df)
df_features = assembler.transform(streaming_df)

# Standardize features for better clustering performance
scaler = StandardScaler(inputCol="raw_features", outputCol="features", withStd=True, withMean=True)
static_scaler_model = scaler.fit(static_features)
static_scaled = static_scaler_model.transform(static_features).select(*feature_cols, "features")
df_scaled = static_scaler_model.transform(df_features).select(*feature_cols, "features")

# Train KMeans model with predefined k (assuming optimal K = 3)
kmeans = KMeans(k=3, seed=1, featuresCol="features")
kmeans_model = kmeans.fit(static_scaled)

# Assign clusters to the streaming data
df_clustered = kmeans_model.transform(df_scaled).select(*feature_cols, "prediction")

# Function to plot streaming clusters
def plot_clusters(batch_df, epoch_id):
    print(f"Looking at batch_df {batch_df.count()}")
    if batch_df.count() > 0:
        pdf = batch_df.toPandas()
        print(pdf)
        # Save plot instead of showing
        plt.figure(figsize=(8, 6))
        plt.scatter(pdf["sepal_length"], pdf["sepal_width"], c=pdf["prediction"], cmap="viridis", alpha=0.6)
        plt.xlabel("Sepal Length")
        plt.ylabel("Sepal Width")
        plt.title(f"Streaming KMeans Clustering - Epoch {epoch_id}")
        plt.colorbar(label="Cluster")

        plot_path = f"/tmp/plot_epoch_{epoch_id}.png"
        plt.savefig(plot_path)
        plt.close()

        print(f"Saved plot: {plot_path}")

# Write streaming results and plot
df_clustered.writeStream.outputMode("append").foreachBatch(plot_clusters).start().awaitTermination()



Downloaded dataset to /tmp/iris.csv
Looking at batch_df 150
     sepal_length  sepal_width  petal_length  petal_width  prediction
0             5.1          3.5           1.4          0.2           1
1             4.9          3.0           1.4          0.2           1
2             4.7          3.2           1.3          0.2           1
3             4.6          3.1           1.5          0.2           1
4             5.0          3.6           1.4          0.2           1
..            ...          ...           ...          ...         ...
145           6.7          3.0           5.2          2.3           0
146           6.3          2.5           5.0          1.9           2
147           6.5          3.0           5.2          2.0           0
148           6.2          3.4           5.4          2.3           0
149           5.9          3.0           5.1          1.8           0

[150 rows x 5 columns]
Saved plot: /tmp/plot_epoch_0.png
